In [10]:
import bs4
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By
import time
import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
webdriver_path=r"C:\Users\hadar\Documents\chromedriver_win32\chromedriver.exe"

In [4]:
def load_soup_object(html_file_name):
    page=requests.get(html_file_name)
    soup=BeautifulSoup(page.content,'html.parser')
    return soup

In [5]:
def load_soup_object_with_file_name(html_file_name):
    page=requests.get(html_file_name)
    soup=BeautifulSoup(page.content,'html.parser')
    return soup

In [6]:
def write_list_to_file(file_name,list_name):
    with open(file_name, 'a') as f:
        for li in list_name:
            f.write("%s\n" % li)

In [7]:
def read_list_from_file(file_name):
    with open(file_name,'r') as f:
        lines = [line.strip() for line in f.readlines()]
    return lines

In [41]:
#scapping articles'urls from the website
#scrapping from multiple pages with 'loas more button' javascript link
#used stackoverflow - https://stackoverflow.com/questions/68792444/how-to-scrape-website-if-it-has-load-more-button-to-load-more-content-on-the-pag
def israel365news_crawler(webdriver_path,max_links):
    driver=webdriver.Chrome(executable_path=webdriver_path)
    soup=load_soup_object('https://www.israel365news.com/israel-news/')

    try:
        driver.maximize_window() #ensures that the window occupies the entire screen.
        driver.implicitly_wait(10) #wait for up to 10 seconds for the element to appear before throwing an exception
        driver.get("https://www.israel365news.com/israel-news/")

        urls=[]

        while max_links>=0:
            if max_links%100==0: print(f'{max_links} urls until finish')
            #articles = driver.find_elements_by_css_selector("h1.elementor-heading-title.elementor-size-default")
            #articles=driver.find_elements("xpath", "//h1[@class='elementor-heading-title elementor-size-default']")
            articles=soup('h1',attrs={"class":"elementor-heading-title elementor-size-default"})

            for art in articles:
                #url=art.find_element_by_tag_name("a").get_attribute("href")
                #url = art.find_element("xpath", "./a").get_attribute("href")
                #urls.append(url)
                #print(f'url appended: {url}')
                a=art.find('a')
                try:
                    if 'href' in a.attrs:
                        url=a.get('href')
                        urls.append(url)
                        #print(f'url appended: {url}')
                        max_links-=1
                except:
                    continue

            #loadmore = driver.find_element_by_css_selector("a.elementor-button-link.elementor-button")
            loadmore=driver.find_element("xpath", "//a[contains(@class, 'elementor-button-link') and contains(@class, 'elementor-button')]")

            if loadmore.is_displayed():
                driver.execute_script("arguments[0].click();", loadmore)
                time.sleep(5)
            else:
                print("no more articles to load")
                break

    except StaleElementReferenceException:
        pass

    finally:
        driver.quit()
    return urls


In [42]:
webdriver_path=r"C:\Users\hadar\Documents\chromedriver_win32\chromedriver.exe"
urls_list=israel365news_crawler(webdriver_path,10000)


C:\Users\hadar\AppData\Local\Temp\ipykernel_6828\2713619630.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(executable_path=webdriver_path)


10000 urls until finish
9900 urls until finish
9800 urls until finish
9700 urls until finish
9600 urls until finish
9500 urls until finish
9400 urls until finish
9300 urls until finish
9200 urls until finish
9100 urls until finish
9000 urls until finish
8900 urls until finish
8800 urls until finish
8700 urls until finish
8600 urls until finish
8500 urls until finish
8400 urls until finish
8300 urls until finish
8200 urls until finish
8100 urls until finish
8000 urls until finish
7900 urls until finish
7800 urls until finish
7700 urls until finish
7600 urls until finish
7500 urls until finish
7400 urls until finish
7300 urls until finish
7200 urls until finish
7100 urls until finish
7000 urls until finish
6900 urls until finish
6800 urls until finish
6700 urls until finish
6600 urls until finish
6500 urls until finish
6400 urls until finish
6300 urls until finish
6200 urls until finish
6100 urls until finish
6000 urls until finish
5900 urls until finish
5800 urls until finish
5700 urls 

OSError: [Errno 22] Invalid argument: 'https://www.israel365news.com/israel-news/'

In [45]:
write_list_to_file('israel365news_urls.txt',urls_list)

In [111]:
#crawl through all articles from Israel365News we've collected
def load_articles_to_df_israel365news(urls_list):
    headline=[]
    writers=[]
    date=[]
    genre=[]
    content=[]
    i=0
    
    for url in urls_list:
        soup=load_soup_object(url)
        headline.append(soup.find("h1",attrs={"class":"elementor-heading-title elementor-size-default"}).get_text().strip())
        writers.append(soup.find("h4",attrs={"class":"elementor-author-box__name"}).get_text().strip())
        date.append(soup.find("div",attrs={"class":"elementor-element elementor-element-8f04d6d elementor-widget__width-auto elementor-widget elementor-widget-heading"}).find("h2").get_text().strip())
        genre.append(", ".join(item.get_text().strip() for item in soup.find("div",attrs={"class":"elementor-element elementor-element-1a40dbf elementor-widget__width-auto text-hover elementor-widget elementor-widget-heading"}).find_all("a")))
        content.append(soup.find("div",attrs={"class":"elementor-element elementor-element-2106a85 elementor-widget elementor-widget-theme-post-content"}).get_text().strip()) 
        i+=1
        if(i%100==0):
            print(f'{i} articles scrapped')

    df=pd.DataFrame({"headline":headline,"writers":writers,"date":date,"genre":genre,"content":content})
    return df

In [112]:
df=load_articles_to_df_israel365news(urls_list)

100 articles scrapped
200 articles scrapped
300 articles scrapped
400 articles scrapped
500 articles scrapped
600 articles scrapped
700 articles scrapped
800 articles scrapped
900 articles scrapped
1000 articles scrapped
1100 articles scrapped
1200 articles scrapped
1300 articles scrapped
1400 articles scrapped
1500 articles scrapped
1600 articles scrapped
1700 articles scrapped
1800 articles scrapped
1900 articles scrapped
2000 articles scrapped
2100 articles scrapped
2200 articles scrapped
2300 articles scrapped
2400 articles scrapped
2500 articles scrapped
2600 articles scrapped
2700 articles scrapped
2800 articles scrapped
2900 articles scrapped
3000 articles scrapped
3100 articles scrapped
3200 articles scrapped
3300 articles scrapped
3400 articles scrapped
3500 articles scrapped
3600 articles scrapped
3700 articles scrapped
3800 articles scrapped
3900 articles scrapped
4000 articles scrapped
4100 articles scrapped
4200 articles scrapped
4300 articles scrapped
4400 articles scrapp

In [113]:
df.to_csv('israel365news.csv', index=False)

In [114]:
df

,headline,writers,date,genre,content
0,Knesset passes 2023-2024 state budget,JNS,"May 24, 2023",Israel News,Israel’s Knesset on Wednesday passed the state...
1,Will flights from Israel to Mecca spark Temple...,Adam Eliyahu Berkowitz,"May 23, 2023",Israel News,Prime Minister Netanyahu was in direct contact...
2,Rabbi Leo Dee weighs $1.3b. suit against Amanpour,JNS,"May 23, 2023",Israel News,Rabbi Leo Dee is considering suing CNN chief i...
3,Israeli Cabinet boosts funds for Kotel infrast...,JNS,"May 22, 2023",Israel News,The Israeli Cabinet on Sunday approved funding...
4,Ben-Gvir visits Temple Mount: ‘We are the owne...,JNS,"May 21, 2023",Israel News,During a visit to the Temple Mount in Jerusale...
...,...,...,...,...,...
10015,Israeli forces kill two terrorists in raid nea...,JNS,"May 10, 2023","Israel News, Judea and Samaria",Israeli forces killed two Palestinian terroris...
10016,IDF eliminates three top Palestinian Islamic J...,JNS,"May 9, 2023","Israel News, Terror Watch",The Israel Defense Forces killed three top Pal...
10017,"After six years, Israel razes illegal PA struc...",JNS,"May 8, 2023",Israel News,Israel’s Civil Administration demolished a Pal...
10018,Jerusalem Day: Mass return to the Temple Mount...,Adam Eliyahu Berkowitz,"May 7, 2023",Israel News,Temple Mount advocacy groups are planning some...


In [169]:
# making crawler for Middle East Monitor website
def load_articles_url_MEM(n_pages):
    soup=load_soup_object(r'https://www.middleeastmonitor.com/category/news/middle-east/')
    articles=soup("li")
    urls=[]
    
    for page in range(n_pages):
        for art in articles:
            a=art.find('a')
            try:
                if 'href' in a.attrs:
                    url=a.get('href')
                    urls.append(url)
            except:
                continue
        next_page=soup.find("a",attrs={"class":"next page-numbers"})["href"]
        soup=load_soup_object(next_page)
        articles=soup("li")
        if(page%50==0): 
            print(f'{page} pages out of 350')
        
    return urls

In [170]:
mem_urls_list=load_articles_url_MEM(350)

0 pages out of 350
50 pages out of 350
100 pages out of 350
150 pages out of 350
200 pages out of 350
250 pages out of 350
300 pages out of 350


In [32]:
#removing links to other sites from list
for url in mem_urls_list:
    if(url.startswith("https://www.middleeastmonitor.com")==False or len(url)<100 or url.find("category")!=-1 or url.find("6-author")!=-1):
        mem_urls_list.remove(url)
write_list_to_file('mem_urls.txt',mem_urls_list)

In [5]:
mem_urls_list=read_list_from_file('mem_urls.txt')

In [13]:
#crawl through all articles from Israel365News we've collected
def load_articles_to_df_mem(urls_list,start_index,max_urls):
    headline=[]
    writers=[]#there is no authors declared inside the articles, will be filled with Unknown
    date=[]
    genre=[]
    content=[]
    for i in range(max_urls):
        try:
            soup=load_soup_object(urls_list[i+start_index])
            he=soup.find("div",attrs={"id":"post-page-title"}).get_text().strip()
            da=soup.find("span",attrs={"class":"post-page-date"}).get_text().strip()
            ge="+".join(item.get_text().strip() for item in soup.find("div",attrs={"class":"post-meta-top"}).find_all("a"))
            co=" ".join(item.get_text().strip() for item in soup.find("div",attrs={"id":"post-content"}).find_all("p"))
            if he and da and ge and co:
                headline.append(he)
                writers.append("Unknown")
                date.append(da)
                genre.append(ge)
                content.append(co)
            if i%100==0:
                print(f'{i} articles scrapped')
            if not (len(headline)==len(writers)==len(date)==len(genre)==len(content)):
                print(f"problam with url {urls_list[i+start_index]}")
                break
        except:
            continue
    print(f'headlines = {len(headline)}, writers={len(writers)}, date = {len(date)}, genre = {len(genre)}, content={len(content)}')
    df=pd.DataFrame({"headline":headline,"writers":writers,"date":date,"genre":genre,"content":content})
    return df

In [8]:
df_mem=load_articles_to_df_mem(mem_urls_list,0,300)

0 articles scrapped
100 articles scrapped
200 articles scrapped
headlines = 300, writers=300, date = 300, genre = 300, content=300


In [9]:
df_mem

,headline,writers,date,genre,content
0,Reporters Without Borders calls for Egypt to r...,Unknown,"May 27, 2023 at 10:37 am",Africa+Egypt+International Organisations+News+...,Reporters Without Borders has called on the Eg...
1,"Russia, Somalia mull establishment of platform...",Unknown,"May 26, 2023 at 8:11 pm",Africa+Europe & Russia+News+Russia+Somalia,"Somali Foreign Minister, Abshir Omar Jama, sai..."
2,Turkiye homes in northern Syria for voluntary ...,Unknown,"May 26, 2023 at 8:07 pm",Europe & Russia+Middle East+News+Syria+Turkey,Ankara is currently working on the safe return...
3,Lebanon to take steps to fix finance sector sh...,Unknown,"May 26, 2023 at 7:59 pm",Africa+Lebanon+Middle East+News,Lebanon will work over the next year to addres...
4,"3 dead, 12 missing as migrant boat capsizes in...",Unknown,"May 26, 2023 at 6:18 pm",Europe & Russia+Greece+News,Three migrants were killed and twelve others a...
...,...,...,...,...,...
295,Lebanon to take steps to fix finance sector sh...,Unknown,"May 26, 2023 at 7:59 pm",Africa+Lebanon+Middle East+News,Lebanon will work over the next year to addres...
296,"3 dead, 12 missing as migrant boat capsizes in...",Unknown,"May 26, 2023 at 6:18 pm",Europe & Russia+Greece+News,Three migrants were killed and twelve others a...
297,Bahrain schools ordered by King to remove less...,Unknown,"May 12, 2023 at 2:59 pm",Asia & Americas+Bahrain+Israel+Middle East+New...,Bahrain's educational institutions have been o...
298,China resumes construction of military base in...,Unknown,"April 30, 2023 at 9:33 am",Asia & Americas+China+Middle East+News+UAE+US,China has resumed construction work on a milit...


In [14]:
df_mem.append(load_articles_to_df_mem(mem_urls_list,1300,5000),ignore_index=True)

0 articles scrapped
100 articles scrapped
200 articles scrapped
300 articles scrapped
400 articles scrapped
500 articles scrapped
600 articles scrapped
700 articles scrapped
800 articles scrapped
900 articles scrapped
1000 articles scrapped
1100 articles scrapped
1200 articles scrapped
1300 articles scrapped
1400 articles scrapped


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


1500 articles scrapped
1600 articles scrapped
1700 articles scrapped
1800 articles scrapped
1900 articles scrapped
2000 articles scrapped
2100 articles scrapped
2200 articles scrapped
2300 articles scrapped
2400 articles scrapped
2500 articles scrapped
2600 articles scrapped
2700 articles scrapped
2800 articles scrapped
2900 articles scrapped
3000 articles scrapped
3100 articles scrapped
3200 articles scrapped
3300 articles scrapped
3400 articles scrapped
3500 articles scrapped
3600 articles scrapped
3700 articles scrapped
3800 articles scrapped
3900 articles scrapped
4000 articles scrapped
4100 articles scrapped
4200 articles scrapped
4300 articles scrapped
headlines = 4312, writers=4312, date = 4312, genre = 4312, content=4312


C:\Users\hadar\AppData\Local\Temp\ipykernel_19508\4023380378.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_mem.append(load_articles_to_df_mem(mem_urls_list,1300,5000),ignore_index=True)


,headline,writers,date,genre,content
0,Reporters Without Borders calls for Egypt to r...,Unknown,"May 27, 2023 at 10:37 am",Africa+Egypt+International Organisations+News+...,Reporters Without Borders has called on the Eg...
1,"Russia, Somalia mull establishment of platform...",Unknown,"May 26, 2023 at 8:11 pm",Africa+Europe & Russia+News+Russia+Somalia,"Somali Foreign Minister, Abshir Omar Jama, sai..."
2,Turkiye homes in northern Syria for voluntary ...,Unknown,"May 26, 2023 at 8:07 pm",Europe & Russia+Middle East+News+Syria+Turkey,Ankara is currently working on the safe return...
3,Lebanon to take steps to fix finance sector sh...,Unknown,"May 26, 2023 at 7:59 pm",Africa+Lebanon+Middle East+News,Lebanon will work over the next year to addres...
4,"3 dead, 12 missing as migrant boat capsizes in...",Unknown,"May 26, 2023 at 6:18 pm",Europe & Russia+Greece+News,Three migrants were killed and twelve others a...
...,...,...,...,...,...
4607,New Israel law to expel Arab students raising ...,Unknown,"May 27, 2023 at 1:15 pm",Israel+Middle East+News+Palestine,The extremist Israeli government is preparing ...
4608,"Israel settlers burn Palestine vehicles, crops...",Unknown,"May 27, 2023 at 1:02 pm",Israel+Middle East+News+Palestine,Extremist Israeli settlers burnt Palestinian v...
4609,"Kuwait, Philippines crisis after workers' visa...",Unknown,"May 27, 2023 at 12:30 pm",Asia & Americas+Kuwait+Middle East+News+Philip...,A new conflict has occurred between Kuwait and...
4610,Algeria ambassador to Italy sparks controversy...,Unknown,"May 27, 2023 at 11:34 am",Africa+Algeria+Europe & Russia+Italy+News+Tuni...,Abdelkrim Touahria has sparked controversy aft...


In [15]:
df_mem.to_csv('mem.csv', index=False)

In [ ]:
#end of scrapping MiddleEast Monitor website
#---------------------------------------------------
#start of scrapping Fox News website

In [26]:
#scapping articles'urls from the website
#scrapping from multiple pages with 'show more' button
def foxnews_url_crawler(webdriver_path,max_links):
    driver=webdriver.Chrome(executable_path=webdriver_path)
    soup=load_soup_object('https://www.foxnews.com/category/world/world-regions/israel')

    try:
        driver.maximize_window() #ensures that the window occupies the entire screen.
        driver.implicitly_wait(10) #wait for up to 10 seconds for the element to appear before throwing an exception
        driver.get("https://www.foxnews.com/category/world/world-regions/israel")

        urls=[]

        while max_links>=0:
            if max_links%100==0: 
                print(f'{max_links} urls until finish')
            articles=soup('article',attrs={"class":"article"})

            for art in articles:
                a=art.find('h4',attrs={"class":"title"}).find('a')
                try:
                    if 'href' in a.attrs:
                        url=a.get('href')
                        urls.append(url)
                        max_links-=1
                except:
                    continue

            showmore=driver.find_element("css selector", "div.button.load-more.js-load-more > a")

            if showmore.is_displayed():
                driver.execute_script("arguments[0].click();", showmore)
                time.sleep(5)
            else:
                print("no more articles to load")
                break

    except StaleElementReferenceException:
        pass
    except NoSuchElementException:
        print("Element not found. No more articles to load.")

    finally:
        driver.quit()
    return urls

In [34]:
fox_url_list=foxnews_url_crawler(webdriver_path,10000)

C:\Users\hadar\AppData\Local\Temp\ipykernel_19508\3621044219.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(executable_path=webdriver_path)


10000 urls until finish
9900 urls until finish
9800 urls until finish
9700 urls until finish
9600 urls until finish
9500 urls until finish
9400 urls until finish
9300 urls until finish
9200 urls until finish
9100 urls until finish
9000 urls until finish
8900 urls until finish
8800 urls until finish
8700 urls until finish
8600 urls until finish
8500 urls until finish
8400 urls until finish
8300 urls until finish
8200 urls until finish
8100 urls until finish
8000 urls until finish
7900 urls until finish
7800 urls until finish
7700 urls until finish
7600 urls until finish
7500 urls until finish
7400 urls until finish
7300 urls until finish
7200 urls until finish
7100 urls until finish
7000 urls until finish
6900 urls until finish
6800 urls until finish
6700 urls until finish
6600 urls until finish
6500 urls until finish
6400 urls until finish
6300 urls until finish
6200 urls until finish
6100 urls until finish
6000 urls until finish
5900 urls until finish
5800 urls until finish
5700 urls 

In [35]:
write_list_to_file('foxnews_urls.txt',fox_url_list)

In [46]:
#crawl through all articles from Fox News we've collected
def load_articles_to_df_foxnews(urls_list):
    headline=[]
    writers=[]
    date=[]
    genre=[]
    content=[]
    i=0
    
    for url in urls_list:
        soup=load_soup_object(r"https://www.foxnews.com/"+url)
        headline.append(soup.find("h1",attrs={"class":"headline"}).get_text().strip())
        writers.append(soup.find("div",attrs={"class":"author-byline"}).find("a").get_text().strip())
        date.append(soup.find("time").get_text().strip())
        genre.append(soup.find("div",attrs={"class":"eyebrow"}).find("a").get_text().strip())
        content.append(" ".join(item.get_text().strip() for item in soup.find("div",attrs={"class":"article-body"}).find_all("p"))) 
        i+=1
        if(i%100==0):
            print(f'{i} articles scrapped')

    df=pd.DataFrame({"headline":headline,"writers":writers,"date":date,"genre":genre,"content":content})
    return df

In [47]:
df_fox=load_articles_to_df_foxnews(fox_url_list)
df_fox

100 articles scrapped
200 articles scrapped
300 articles scrapped
400 articles scrapped
500 articles scrapped
600 articles scrapped
700 articles scrapped
800 articles scrapped
900 articles scrapped
1000 articles scrapped
1100 articles scrapped
1200 articles scrapped
1300 articles scrapped
1400 articles scrapped
1500 articles scrapped
1600 articles scrapped
1700 articles scrapped
1800 articles scrapped
1900 articles scrapped
2000 articles scrapped
2100 articles scrapped
2200 articles scrapped
2300 articles scrapped
2400 articles scrapped
2500 articles scrapped
2600 articles scrapped
2700 articles scrapped
2800 articles scrapped
2900 articles scrapped
3000 articles scrapped
3100 articles scrapped
3200 articles scrapped
3300 articles scrapped
3400 articles scrapped
3500 articles scrapped
3600 articles scrapped
3700 articles scrapped
3800 articles scrapped
3900 articles scrapped
4000 articles scrapped
4100 articles scrapped
4200 articles scrapped
4300 articles scrapped
4400 articles scrapp

,headline,writers,date,genre,content
0,"Jewish groups, allies demand CUNY Law lose fun...",Yael Halon,"May 30, 2023 5:00am EDT",Media,Filmmaker Steven Spielberg warned against anti...
1,Massive sea urchin die-off in Israeli gulf thr...,Associated Press,"May 27, 2023 10:00am EDT",Israel,Sea otters use rocks as tools to crack tough s...
2,Backlash against Roger Waters for antisemitism...,Lindsay Kornick,"May 26, 2023 11:23am EDT",Media,Pink Floyd rocker Roger Waters tells Richard B...
3,"Iran regime close to getting nuclear bomb, but...",Benjamin Weinthal,"May 26, 2023 2:00am EDT",Iran,Fox News' Jennifer Griffin reports from the Pe...
4,White House unveils first-ever national strate...,Aaron Kliegman,"May 25, 2023 5:17pm EDT",Antisemitism,Filmmaker Steven Spielberg warned against anti...
...,...,...,...,...,...
6105,Israel's defense chief warns Tehran against fu...,Caitlin McFall,"May 23, 2023 10:53am EDT",Iran,U.S. Central Command and the IDF took part in ...
6106,CNN’s Christine Amanpour apologizes after wron...,Jeffrey Clark,"May 23, 2023 10:42am EDT",Media,CNN chief international correspondent Christia...
6107,Israel more than doubled strikes on Iranian ta...,Associated Press,"May 22, 2023 8:07pm EDT",Israel,Fox News Flash top headlines are here. Check o...
6108,Iran building nuclear facility deep enough tha...,Michael Lee,"May 22, 2023 1:31pm EDT",Iran,Exiled Crown Prince of Iran Reza Pahlavi joine...


In [48]:
df_fox.to_csv('fox.csv', index=False)

In [ ]:
#scrapping frpm nbc News website all articles' urls
def scrapping_all_urls_from_nbc_news(url_1)
    urls = []
    specific_urls = ["politics", "us-news", "world", "business", "tech-media", "health", "culture-matters"]

    for url_part in specific_urls:
        base_url = url_1 + url_part

        webdriver_path = r"C:\Users\ofir\Downloads\chromedriver_win32\chromedriver.exe"
        driver = webdriver.Chrome(executable_path=webdriver_path)
        driver.maximize_window()
        driver.implicitly_wait(10)
        driver.get(base_url)

        try:
            while True:
                loadmore = driver.find_element(By.CSS_SELECTOR, 'button.animated-ghost-button')
                if not loadmore.is_displayed():
                    break
                loadmore.click()
                time.sleep(10)
                soup = load_soup_object_with_file_content(driver.page_source)
                articles = soup.find_all('div', attrs={"class": "wide-tease-item__info-wrapper flex-grow-1-m"})
                for article in articles:
                    try:
                        a_elements = article.find_all('a')[-1]
                        if 'href' in a_elements.attrs:
                            url = a_elements.get('href')
                            if url not in urls:  
                                urls.append(url)
                    except:
                        print("exception in articles loop")

        except StaleElementReferenceException:
            pass
        except  NoSuchElementException:
            pass

    driver.quit()
    return urls

In [ ]:
#crawler for nbc News website
def load_articles_to_df_nbcnews(urls_list):
    headline=[]
    writers=[]
    date=[]
    genre=[]
    content=[]
    i=0
    for url in urls_list:
        soup = load_soup_object_with_file_name(url)
        try:
            headline.append(soup.find("div",attrs={"class":"article-hero-headline layout-grid-item grid-col-10-l"}).find("h1").get_text().strip())
            writers.append(soup.find("span",attrs={"class":"byline-name"}).get_text().strip())
            date.append(soup.find("time",attrs={"class":"relative z-1"}).get_text().strip())   
            genre.append(soup.find("div",attrs={"class":"unibrow articleTitleSection article-hero__tax-term"}).get_text().strip())    
            content_element = soup.find("div", class_="article-body__content")
            if content_element:
                paragraphs = content_element.find_all("p")
                content1 = ""
                for paragraph in paragraphs:
                # Break the loop if a specific condition is met, such as finding a specific HTML line or tag
                    if paragraph.find("top_share_widget"):
                        break
                    content1 += paragraph.get_text().strip() + " "  
                content.append(content1)
            else:
                content.append("None")
            i+=1
            if(i%100==0):
                print(f'{i} articles scrapped')
        except:
            print("error in link: " + url)
    df= pd.DataFrame({"headline":headline, "writers":writers,"date":date,"genre":genre,"content":content})
    return df

In [ ]:
url_1 = "https://www.nbcnews.com/"
urls = scrapping_all_urls_from_nbc_news(url_1)
df1 = load_articles_to_df_nbcnews(urls)
df1.to_csv('nbcnews.csv', index=False)
df1

In [ ]:
####
#--------------------
###crawler for Empire new - Fake news website

In [ ]:
def scrapping_all_urls_from_empire_news(base_url)    
    
    urls = []

    for page in range(1,280):
        pages_list = base_url.format(page)
        response = requests.get(pages_list)
        soup = BeautifulSoup(response.content,"html.parser")
        articles=soup('div',attrs={"class":"span6 home-post"})

        for art in articles:
            a=art.find('a')
            try:
                if 'href' in a.attrs:
                    url=a.get('href')
                    urls.append(url)
            except:
                continue
    return urls

In [ ]:
def load_articles_to_df_empirenews(urls_list):
    headline=[]
    writers=[]
    date=[]
    genre=[]
    content=[]
    i=0
    for url in urls_list:
        try:
            soup=load_soup_object_with_file_name(url)
            headline.append(soup.find("h1",attrs={"class":"entry-title"}).get_text().strip())
            writers.append(soup.find("span",attrs={"class":"author vcard"}).get_text().strip())
            date.append(soup.find("time",attrs={"class":"entry-date published updated"}).get_text().strip())   
            genre.append(soup.find("span",attrs={"class":"cat-links"}).get_text().strip())    
            content_element = soup.find("div", class_="entry-content")
            if content_element:
                paragraphs = content_element.find_all("p")
                content1 = ""
                for paragraph in paragraphs:
                # Break the loop if a specific condition is met, such as finding a specific HTML line or tag
                    if paragraph.find("top_share_widget"):
                        break
                    content1 += paragraph.get_text().strip() + " "  
                content.append(content1)
            else:
                content.append("None")
            i+=1
            if(i%100==0):
                print(f'{i} articles scrapped')
        except:
            continue
            
    df= pd.DataFrame({"headline":headline,"writers":writers,"date":date,"genre":genre,"content":content})
    return df

In [ ]:
base_url = "https://empirenews.net/page/{}/"
urls = scrapping_all_urls_from_empire_news(base_url)    
my_df = load_articles_to_df_empirenews(urls)
my_df.to_csv('empirenews.csv',index=False)
my_df

In [14]:
##############
#Data Preprocessing
##############

In [15]:
#Israel 365 News
israel_news_df=pd.read_csv('israel365news.csv')
israel_news_df

,headline,writers,date,genre,content
0,Knesset passes 2023-2024 state budget,JNS,"May 24, 2023",Israel News,Israel’s Knesset on Wednesday passed the state...
1,Will flights from Israel to Mecca spark Temple...,Adam Eliyahu Berkowitz,"May 23, 2023",Israel News,Prime Minister Netanyahu was in direct contact...
2,Rabbi Leo Dee weighs $1.3b. suit against Amanpour,JNS,"May 23, 2023",Israel News,Rabbi Leo Dee is considering suing CNN chief i...
3,Israeli Cabinet boosts funds for Kotel infrast...,JNS,"May 22, 2023",Israel News,The Israeli Cabinet on Sunday approved funding...
4,Ben-Gvir visits Temple Mount: ‘We are the owne...,JNS,"May 21, 2023",Israel News,During a visit to the Temple Mount in Jerusale...
...,...,...,...,...,...
10015,Israeli forces kill two terrorists in raid nea...,JNS,"May 10, 2023","Israel News, Judea and Samaria",Israeli forces killed two Palestinian terroris...
10016,IDF eliminates three top Palestinian Islamic J...,JNS,"May 9, 2023","Israel News, Terror Watch",The Israel Defense Forces killed three top Pal...
10017,"After six years, Israel razes illegal PA struc...",JNS,"May 8, 2023",Israel News,Israel’s Civil Administration demolished a Pal...
10018,Jerusalem Day: Mass return to the Temple Mount...,Adam Eliyahu Berkowitz,"May 7, 2023",Israel News,Temple Mount advocacy groups are planning some...


In [16]:
israel_news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10020 entries, 0 to 10019
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   headline  10020 non-null  object
 1   writers   10020 non-null  object
 2   date      10020 non-null  object
 3   genre     10020 non-null  object
 4   content   10020 non-null  object
dtypes: object(5)
memory usage: 391.5+ KB


In [17]:
#changing date format
israel_news_df['date']=pd.to_datetime(israel_news_df['date'])

In [40]:
#add is_fake column and set to nan - a source associated with misinformation that we want to check with our machine
israel_news_df['is_fake']=np.nan
israel_news_df

,headline,writers,date,genre,content,is_fake
0,Knesset passes 2023-2024 state budget,JNS,2023-05-24,Israel News,Israel’s Knesset on Wednesday passed the state...,NaN
1,Will flights from Israel to Mecca spark Temple...,Adam Eliyahu Berkowitz,2023-05-23,Israel News,Prime Minister Netanyahu was in direct contact...,NaN
2,Rabbi Leo Dee weighs $1.3b. suit against Amanpour,JNS,2023-05-23,Israel News,Rabbi Leo Dee is considering suing CNN chief i...,NaN
3,Israeli Cabinet boosts funds for Kotel infrast...,JNS,2023-05-22,Israel News,The Israeli Cabinet on Sunday approved funding...,NaN
4,Ben-Gvir visits Temple Mount: ‘We are the owne...,JNS,2023-05-21,Israel News,During a visit to the Temple Mount in Jerusale...,NaN
...,...,...,...,...,...,...
10015,Israeli forces kill two terrorists in raid nea...,JNS,2023-05-10,"Israel News, Judea and Samaria",Israeli forces killed two Palestinian terroris...,NaN
10016,IDF eliminates three top Palestinian Islamic J...,JNS,2023-05-09,"Israel News, Terror Watch",The Israel Defense Forces killed three top Pal...,NaN
10017,"After six years, Israel razes illegal PA struc...",JNS,2023-05-08,Israel News,Israel’s Civil Administration demolished a Pal...,NaN
10018,Jerusalem Day: Mass return to the Temple Mount...,Adam Eliyahu Berkowitz,2023-05-07,Israel News,Temple Mount advocacy groups are planning some...,NaN


In [22]:
#Fox News
fox_df=pd.read_csv('fox.csv')
fox_df

,headline,writers,date,genre,content
0,"Jewish groups, allies demand CUNY Law lose fun...",Yael Halon,"May 30, 2023 5:00am EDT",Media,Filmmaker Steven Spielberg warned against anti...
1,Massive sea urchin die-off in Israeli gulf thr...,Associated Press,"May 27, 2023 10:00am EDT",Israel,Sea otters use rocks as tools to crack tough s...
2,Backlash against Roger Waters for antisemitism...,Lindsay Kornick,"May 26, 2023 11:23am EDT",Media,Pink Floyd rocker Roger Waters tells Richard B...
3,"Iran regime close to getting nuclear bomb, but...",Benjamin Weinthal,"May 26, 2023 2:00am EDT",Iran,Fox News' Jennifer Griffin reports from the Pe...
4,White House unveils first-ever national strate...,Aaron Kliegman,"May 25, 2023 5:17pm EDT",Antisemitism,Filmmaker Steven Spielberg warned against anti...
...,...,...,...,...,...
6105,Israel's defense chief warns Tehran against fu...,Caitlin McFall,"May 23, 2023 10:53am EDT",Iran,U.S. Central Command and the IDF took part in ...
6106,CNN’s Christine Amanpour apologizes after wron...,Jeffrey Clark,"May 23, 2023 10:42am EDT",Media,CNN chief international correspondent Christia...
6107,Israel more than doubled strikes on Iranian ta...,Associated Press,"May 22, 2023 8:07pm EDT",Israel,Fox News Flash top headlines are here. Check o...
6108,Iran building nuclear facility deep enough tha...,Michael Lee,"May 22, 2023 1:31pm EDT",Iran,Exiled Crown Prince of Iran Reza Pahlavi joine...


In [23]:
fox_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6110 entries, 0 to 6109
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   headline  6110 non-null   object
 1   writers   6110 non-null   object
 2   date      6110 non-null   object
 3   genre     6110 non-null   object
 4   content   6110 non-null   object
dtypes: object(5)
memory usage: 238.8+ KB


In [26]:
#changing date format
fox_df['date']=pd.to_datetime(fox_df['date']).dt.normalize()

In [36]:
#add is_fake column and set to zero - reliable website
fox_df['is_fake']=0
fox_df

,headline,writers,date,genre,content,is_fake
0,"Jewish groups, allies demand CUNY Law lose fun...",Yael Halon,2023-05-30,Media,Filmmaker Steven Spielberg warned against anti...,0
1,Massive sea urchin die-off in Israeli gulf thr...,Associated Press,2023-05-27,Israel,Sea otters use rocks as tools to crack tough s...,0
2,Backlash against Roger Waters for antisemitism...,Lindsay Kornick,2023-05-26,Media,Pink Floyd rocker Roger Waters tells Richard B...,0
3,"Iran regime close to getting nuclear bomb, but...",Benjamin Weinthal,2023-05-26,Iran,Fox News' Jennifer Griffin reports from the Pe...,0
4,White House unveils first-ever national strate...,Aaron Kliegman,2023-05-25,Antisemitism,Filmmaker Steven Spielberg warned against anti...,0
...,...,...,...,...,...,...
6105,Israel's defense chief warns Tehran against fu...,Caitlin McFall,2023-05-23,Iran,U.S. Central Command and the IDF took part in ...,0
6106,CNN’s Christine Amanpour apologizes after wron...,Jeffrey Clark,2023-05-23,Media,CNN chief international correspondent Christia...,0
6107,Israel more than doubled strikes on Iranian ta...,Associated Press,2023-05-22,Israel,Fox News Flash top headlines are here. Check o...,0
6108,Iran building nuclear facility deep enough tha...,Michael Lee,2023-05-22,Iran,Exiled Crown Prince of Iran Reza Pahlavi joine...,0


In [62]:
#Empire News
empirenews_df = pd.read_csv('empirenews.csv')

#remove the "posted in" words
empirenews_df['genre'] = empirenews_df['genre'].str.replace('Posted in', '')

#changing the date format
empirenews_df['date'] = pd.to_datetime(empirenews_df['date']).dt.normalize()

#add is_fake column and set to 1 - fake news website
empirenews_df['is_fake']=1
empirenews_df

,headline,writers,date,genre,content,is_fake
0,Only Hours After Announcing Separation from Li...,Bob The Empire News Potato,2022-01-14,"Entertainment, Headlines","NEW YORK CITY, New York – Only a few hours aft...",1
1,Former President Trump Calls Kyle Rittenhouse ...,Bob The Empire News Potato,2021-11-24,"Headlines, Law, Politics","WASHINGTON, D.C. – A leaked private recording ...",1
2,Kraft Foods To Re-Brand ‘Kool-Aid Man’ Mascot ...,Bob The Empire News Potato,2021-11-24,Business,"CHICAGO, Illinois – Kraft Food Holdings, the p...",1
3,"Multiple People Hospitalized, Several Arrested...",Bob The Empire News Potato,2020-02-08,"Headlines, Law","BOSTON, MA Over a dozen people were hospitali...",1
4,Trump To Host Massive ‘I Beat Impeachment’ Par...,Bob The Empire News Potato,2020-02-08,"Government, Headlines","WASHINGTON, D.C. President Trump has decided ...",1
...,...,...,...,...,...,...
2124,George W. Bush Arrested for Cocaine Possession,JP,2014-07-24,"Headlines, Politics","DALLAS, Texas – One of the most notorious Pres...",1
2125,"Body Found Under Motel Bed, Police Say It Has ...",JP,2014-07-24,"Headlines, Law","MOUNT LAUREL, New Jersey – Stunning news this ...",1
2126,5-Year-Old Finds Human Finger In His McDonald’...,JP,2014-07-23,"Business, Headlines","MONTPELIER, Vermont – One of the biggest fast ...",1
2127,Lindsay Lohan Blames Poor Life Decisions On Zs...,Mange,2014-07-23,Entertainment,"HOLLYWOOD, California – Once again in the news...",1


In [67]:
#nbc News
nbcnews_df = pd.read_csv('nbcnews.csv')


# changing the date format-
# Define the regular expressions for both formats
format1_regex = r'\w+ \d+, \d{4}, \d{1,2}:\d{2} [AP]M UTC'
format2_regex = r'Updated \w+ \d+, \d{4} \d{1,2}:\d{2} [AP]M UTC'

# Extract dates based on the regular expressions
nbcnews_df['date'] = nbcnews_df['date'].str.extract(f'({format1_regex}|{format2_regex})')

# Convert to datetime
nbcnews_df['date'] = pd.to_datetime(nbcnews_df['date'], format='%B %d, %Y, %I:%M %p UTC').dt.normalize()


#add is_fake column and set to 0 - reliable news website
nbcnews_df['is_fake']=0
nbcnews_df.info()
nbcnews_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 807 entries, 0 to 806
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   headline  807 non-null    object        
 1   writers   807 non-null    object        
 2   date      807 non-null    datetime64[ns]
 3   genre     807 non-null    object        
 4   content   807 non-null    object        
 5   is_fake   807 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 38.0+ KB


,headline,writers,date,genre,content,is_fake
0,YouTube will stop removing false claims about ...,"Jennifer Elias, CNBC",2023-06-02,Tech News,Google-owned YouTube will stop removing false ...,0
1,Immigration backlog has a U.S. asylum-seeker f...,"Anagilmara Vilchez, Noticias Telemundo",2023-06-02,Immigration,"MIAMI — Although he's free, the 26-year-old ma...",0
2,GOP requires pledge to back nominee for first ...,Ben Kamisar,2023-06-02,2024 Election,The Republican National Committee is requiring...,0
3,Report finds more than 340 migrant kids were s...,Laura Strickler,2023-06-02,Immigration,The Biden administration released 344 unaccomp...,0
4,FBI to show House Oversight leaders document i...,Rose Horowitch,2023-06-02,Congress,The FBI will let the chair and ranking member ...,0
...,...,...,...,...,...,...
802,Adam Sandler takes home Mark Twain Prize for A...,The Associated Press,2023-03-20,Culture Matters,WASHINGTON — Adam Sandler placed his hand on t...,0
803,Jason Sudeikis and 'Ted Lasso' cast members to...,Peter Alexander,2023-03-19,Culture Matters,"The cast of hit show ""Ted Lasso"" is expected t...",0
804,'Jurassic Park’ star Sam Neill reveals he was ...,"Chrissy Callahan, TODAY",2023-03-18,Culture Matters,While Sam Neill was promoting “Jurassic World ...,0
805,"After hearing of Biden’s potential TikTok ban,...",Kalhan Rosenblatt,2023-03-17,Culture Matters,Their favorite app might get banned — but inst...,0


In [91]:
#MiddleEast Monitor website
mem_df=pd.read_csv('mem.csv')

#change date column to datetime type
mem_df['date']=pd.to_datetime(mem_df['date']).dt.normalize()
mem_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   headline  300 non-null    object        
 1   writers   300 non-null    object        
 2   date      300 non-null    datetime64[ns]
 3   genre     300 non-null    object        
 4   content   300 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 11.8+ KB


In [92]:
mem_df['genre'].value_counts()

Israel+Middle East+News+Palestine                                                        47
Europe & Russia+Middle East+News+Syria+Turkey                                            16
Africa+Lebanon+Middle East+News                                                          10
Africa+Egypt+International Organisations+News+Reporters Without Borders (RSF)+UN          9
Africa+Europe & Russia+News+Russia+Somalia                                                9
                                                                                         ..
Europe & Russia+Germany+Lebanon+Middle East+News                                          1
Asia & Americas+HRW+International Organisations+Middle East+News+Syria+Tajikistan+UN      1
IMF+International Organisations+Lebanon+Middle East+News                                  1
Africa+Bahrain+International Organisations+Jordan+Lebanon+Middle East+News+Tunisia+UN     1
Bahrain+GCC+International Organisations+Middle East+News+Saudi Arabia           

In [93]:
#Define the priority list of genres
priority_list=['Middle East','Europe & Russia','Americas','International Organisations','Israel']

#Function to select the genre based on the priority list
def select_genre(genre_list):
    for genre in priority_list:
        if genre in genre_list:
            return genre
    return 'News'

In [94]:
mem_df['genre']=mem_df['genre'].apply(select_genre)

In [95]:
#add is_fake column and set to nan - a source associated with misinformation that we want to check with our machine
mem_df['is_fake']=np.nan
mem_df

,headline,writers,date,genre,content,is_fake
0,Reporters Without Borders calls for Egypt to r...,Unknown,2023-05-27,International Organisations,Reporters Without Borders has called on the Eg...,NaN
1,"Russia, Somalia mull establishment of platform...",Unknown,2023-05-26,Europe & Russia,"Somali Foreign Minister, Abshir Omar Jama, sai...",NaN
2,Turkiye homes in northern Syria for voluntary ...,Unknown,2023-05-26,Middle East,Ankara is currently working on the safe return...,NaN
3,Lebanon to take steps to fix finance sector sh...,Unknown,2023-05-26,Middle East,Lebanon will work over the next year to addres...,NaN
4,"3 dead, 12 missing as migrant boat capsizes in...",Unknown,2023-05-26,Europe & Russia,Three migrants were killed and twelve others a...,NaN
...,...,...,...,...,...,...
295,Lebanon to take steps to fix finance sector sh...,Unknown,2023-05-26,Middle East,Lebanon will work over the next year to addres...,NaN
296,"3 dead, 12 missing as migrant boat capsizes in...",Unknown,2023-05-26,Europe & Russia,Three migrants were killed and twelve others a...,NaN
297,Bahrain schools ordered by King to remove less...,Unknown,2023-05-12,Middle East,Bahrain's educational institutions have been o...,NaN
298,China resumes construction of military base in...,Unknown,2023-04-30,Middle East,China has resumed construction work on a milit...,NaN


In [102]:
merged_df = pd.concat([israel_news_df, fox_df, empirenews_df, nbcnews_df, mem_df], ignore_index=True)
merged_df

,headline,writers,date,genre,content,is_fake
0,Knesset passes 2023-2024 state budget,JNS,2023-05-24,Israel News,Israel’s Knesset on Wednesday passed the state...,NaN
1,Will flights from Israel to Mecca spark Temple...,Adam Eliyahu Berkowitz,2023-05-23,Israel News,Prime Minister Netanyahu was in direct contact...,NaN
2,Rabbi Leo Dee weighs $1.3b. suit against Amanpour,JNS,2023-05-23,Israel News,Rabbi Leo Dee is considering suing CNN chief i...,NaN
3,Israeli Cabinet boosts funds for Kotel infrast...,JNS,2023-05-22,Israel News,The Israeli Cabinet on Sunday approved funding...,NaN
4,Ben-Gvir visits Temple Mount: ‘We are the owne...,JNS,2023-05-21,Israel News,During a visit to the Temple Mount in Jerusale...,NaN
...,...,...,...,...,...,...
19361,Lebanon to take steps to fix finance sector sh...,Unknown,2023-05-26,Middle East,Lebanon will work over the next year to addres...,NaN
19362,"3 dead, 12 missing as migrant boat capsizes in...",Unknown,2023-05-26,Europe & Russia,Three migrants were killed and twelve others a...,NaN
19363,Bahrain schools ordered by King to remove less...,Unknown,2023-05-12,Middle East,Bahrain's educational institutions have been o...,NaN
19364,China resumes construction of military base in...,Unknown,2023-04-30,Middle East,China has resumed construction work on a milit...,NaN


In [103]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19366 entries, 0 to 19365
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   headline  19366 non-null  object        
 1   writers   19366 non-null  object        
 2   date      19366 non-null  datetime64[ns]
 3   genre     19366 non-null  object        
 4   content   19366 non-null  object        
 5   is_fake   9046 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 907.9+ KB


In [110]:
merged_df.describe(include='all') 

C:\Users\hadar\AppData\Local\Temp\ipykernel_4176\1478616577.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  merged_df.describe(include='all')


,headline,writers,date,genre,content,is_fake
count,19366,19366,19366,19366,19366,9046.000000
unique,3165,241,662,250,3165,NaN
top,"Iran regime close to getting nuclear bomb, but...",JNS,2023-05-23 00:00:00,Israel,Fox News' Jennifer Griffin reports from the Pe...,NaN
freq,611,8517,2272,11242,611,NaN
first,NaN,NaN,2014-07-23 00:00:00,NaN,NaN,NaN
last,NaN,NaN,2023-06-03 00:00:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,0.235353
std,NaN,NaN,NaN,NaN,NaN,0.424242
min,NaN,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,NaN,0.000000


In [111]:
#data cleaning
def remove_duplicatives(df, col_name=None):
    df_copy=df.copy()
    if(col_name==None):
        return df_copy.drop_duplicates()
    return df_copy.drop_duplicates(subset=[col_name])

def remove_corrupt_rows(df, num_max_missing_cols):
    df_corrupt=df.copy()
    non_na=df.shape[1]-num_max_missing_cols
    return df_corrupt.dropna(axis=0,thresh=non_na)



In [ ]:
df['genre'] = df['genre'].str.lower()


In [104]:
#genre column
merged_df['genre'].value_counts()

Israel News                                           7014
Media                                                 1833
Iran                                                  1833
Israel                                                1222
Israel News, Terror Watch                             1002
                                                      ... 
 Business, Headlines, Law, Politics, World               1
 Business, Entertainment, Law, Religion                  1
 Business, Politics, Religion                            1
 Business, Health/Fitness, Politics, Uncategorized       1
 Uncategorized, Weird News                               1
Name: genre, Length: 295, dtype: int64

In [107]:
merged_df.loc[merged_df['genre'].str.contains('Israel'),'genre']='Israel'
merged_df.loc[merged_df['genre'].str.contains('Entertainment'),'genre']='Entertainment'
merged_df

,headline,writers,date,genre,content,is_fake
0,Knesset passes 2023-2024 state budget,JNS,2023-05-24,Israel,Israel’s Knesset on Wednesday passed the state...,NaN
1,Will flights from Israel to Mecca spark Temple...,Adam Eliyahu Berkowitz,2023-05-23,Israel,Prime Minister Netanyahu was in direct contact...,NaN
2,Rabbi Leo Dee weighs $1.3b. suit against Amanpour,JNS,2023-05-23,Israel,Rabbi Leo Dee is considering suing CNN chief i...,NaN
3,Israeli Cabinet boosts funds for Kotel infrast...,JNS,2023-05-22,Israel,The Israeli Cabinet on Sunday approved funding...,NaN
4,Ben-Gvir visits Temple Mount: ‘We are the owne...,JNS,2023-05-21,Israel,During a visit to the Temple Mount in Jerusale...,NaN
...,...,...,...,...,...,...
19361,Lebanon to take steps to fix finance sector sh...,Unknown,2023-05-26,Middle East,Lebanon will work over the next year to addres...,NaN
19362,"3 dead, 12 missing as migrant boat capsizes in...",Unknown,2023-05-26,Europe & Russia,Three migrants were killed and twelve others a...,NaN
19363,Bahrain schools ordered by King to remove less...,Unknown,2023-05-12,Middle East,Bahrain's educational institutions have been o...,NaN
19364,China resumes construction of military base in...,Unknown,2023-04-30,Middle East,China has resumed construction work on a milit...,NaN


In [109]:
merged_df['genre'].value_counts()

Israel                                                11242
Media                                                  1833
Iran                                                   1833
Antisemitism                                            611
Digging History                                         611
                                                      ...  
 Headlines, Politics, Religion, World                     1
 Business, Headlines, Weird News, World                   1
 Business, Politics, Science/Tech, Weird News             1
 Government, Headlines, Health/Fitness, Weird News        1
 Business, Law, Videos                                    1
Name: genre, Length: 250, dtype: int64

In [ ]:
priority_list=['Middle East','Europe & Russia','Americas','International Organisations','Israel','Politics', '']